In [1]:
import requests
import pandas as pd
import numpy as np
import time

In [2]:
access_token = 'Your access token'

In [3]:
df = pd.read_csv('E:\ABD-work\Sample_data\sample_last_2021.txt' , sep = '^' , low_memory=False)
df.head()

,Enumeration_date,NPI,Sole_proprietor,Taxonomies_code,Taxonomies_desc,First_name,Last_name,Mid_name,Location_country_code,Location_address1,...,Location_telephone_number,Location_fax_number,Mailing_country_code,Mailing_address1,Mailing_address2,Mailing_city,Mailing_state,Mailing_postal_code,Mailing_telephone_number,Mailing_fax_number
0,2021-10-26,1700540606,NO,163W00000X,Registered Nurse,PAYAL,PATEL,T,US,4480 UTICA RIDGE RD STE 1140,...,563-742-5700,563-742-5705,US,4480 UTICA RIDGE RD STE 1140,NaN,BETTENDORF,IA,527221644.0,563-742-5700,563-742-5705
1,2021-10-26,1700540762,NO,101Y00000X,Counselor,KAITLYN,MCCLUNG,NaN,US,320 WESTWAY PL STE 530,...,817-916-9100,NaN,US,320 WESTWAY PL STE 530,NaN,ARLINGTON,TX,760181000.0,817-916-9100,NaN
2,2021-10-26,1720742711,NO,225100000X,Physical Therapist,AASHINI,PATEL,NaN,US,6501 S CONGRESS AVE STE 301,...,512-270-2060,NaN,US,7505 N LOOP 1604 E STE 101,NaN,LIVE OAK,TX,782332604.0,210-590-4000,210-590-4585
3,2021-10-26,1720742729,YES,363LF0000X,"Nurse Practitioner, Family",IVAN,VEGA,HORACIO,US,151 S OAK AVE STE 7,...,928-366-5166,928-366-5165,US,PO BOX 617,NaN,SOMERTON,AZ,853500617,928-315-7910,928-722-6113
4,2021-10-26,1730843798,NO,106S00000X,Behavior Technician,MEGAN,HUNT,NaN,US,1353 E MAIN ST,...,317-520-4748,NaN,US,1353 E MAIN ST,NaN,BROWNSBURG,IN,461121433.0,317-520-4748,NaN


In [4]:
df.shape

(484812, 24)

In [5]:
data_splitted = np.array_split(df, 40)

In [6]:
df['OrcidID_Exact'] = np.nan
df['OrcidID_Fuzzy'] = np.nan

In [7]:
df['Match_Type'] = np.nan
df['NumOfResults'] = np.nan

In [8]:
# Set the API endpoint

api_endpoint = "https://pub.orcid.org/v3.0/search"

In [9]:
# Set the authorization header and payload
headers = {
    "Authorization": f"Bearer {access_token}",
    "Accept": "application/json",
}

In [10]:
params = {}
exact_count = 0
fuzzy_count = 0
too_fuzzy_count = 0
else_count = 0
no_name_count = 0

In [ ]:
df.isna().sum()

In [14]:
# data_splitted[0]

In [ ]:
for index, row in data_splitted[0].iterrows():
    given_name = row['First_name']
    family_name = row['Last_name']
    middle_name = row['Mid_name']
    # affiliation = row['affiliation']
    # email = row['C16: email']
    # orcid = '0000-0002-7419-0850'
    # name = 'William Collins'

    # params = {
    #     # 'q' : A full text search query that searches across multiple fields in the ORCID record, including the researcher's name, title, abstract, and keywords.
    #     "q": f"family-name:{family_name} AND given-names:{given_name} affiliation-org-name:{affiliation} email:{email} credit-name:{middle_name} other-names:{middle_name}" ,
    # }
    
    params = {
        "q": f"family-name:{family_name} AND given-names:{given_name} other-names:{middle_name}" ,
    }
        
    response = requests.get(api_endpoint, headers=headers, params=params)
    results = response.json()
        
    # Check the status code of the response
    if response.status_code == 200:
        # If the request is successful, the search results will be in the response
        if results["num-found"] == 1:
            # Get the only ORCID iD holder from the search results
            orcid_id = results["result"][0]["orcid-identifier"]["path"]
            # row['OrcidID'] = orcid_id # Burası kalıcı olmuyor buna bak.
            df.at[index,'OrcidID_Exact'] = orcid_id
            df.at[index,'Match_Type'] = 'Exact'
            df.at[index,'NumOfResults'] = 1
            exact_count += 1
            # print(f"Exact ORCID iD: {orcid_id}")
        elif 1 < results["num-found"] <= 5:
            # Get the first ORCID iD holder from the search results
            # orcid_id = results["result"][0]["orcid-identifier"]["path"]
            # row['OrcidID'] = orcid_id
            orcid_ids = []
            for i in range(results["num-found"]):
                orcid_id = results["result"][i]["orcid-identifier"]["path"]
                orcid_ids.append(orcid_id)
                
                
            df.at[index,'OrcidID_Fuzzy'] = orcid_ids
            df.at[index,'Match_Type'] = 'Fuzzy'
            df.at[index,'NumOfResults'] = results["num-found"]
            fuzzy_count += 1
        elif 5 < results["num-found"] :
            # Get the first ORCID iD holder from the search results
            # orcid_id = results["result"][0]["orcid-identifier"]["path"]
            # row['OrcidID'] = orcid_id
            df.at[index,'OrcidID_Fuzzy'] = np.nan
            df.at[index,'Match_Type'] = 'Too_Fuzzy'
            df.at[index,'NumOfResults'] = results["num-found"]
            too_fuzzy_count += 1
        else:
            no_name_count += 1
    else:
        else_count += 1
        print('Error Message !')
        
    time.sleep(2)
            

In [ ]:
# for index, row in df.iterrows():
#     if type(row['basic_first_name0'] != float) :
#         given_name = row['basic_first_name0']
#         family_name = row['basic_last_name0']
#         middle_name = row['basic_middle_name0']
#         # affiliation = row['affiliation']
#         # email = row['C16: email']
#         # orcid = '0000-0002-7419-0850'
#         # name = 'William Collins'

#         # params = {
#         #     # 'q' : A full text search query that searches across multiple fields in the ORCID record, including the researcher's name, title, abstract, and keywords.
#         #     "q": f"family-name:{family_name} AND given-names:{given_name} affiliation-org-name:{affiliation} email:{email} credit-name:{middle_name} other-names:{middle_name}" ,
#         # }
        
#         params = {
#             "q": f"family-name:{family_name} AND given-names:{given_name} other-names:{middle_name}" ,
#         }
            
#         response = requests.get(api_endpoint, headers=headers, params=params)
#         results = response.json()
            
#         # Check the status code of the response
#         if response.status_code == 200:
#             # If the request is successful, the search results will be in the response
#             if results["num-found"] == 1:
#                 # Get the only ORCID iD holder from the search results
#                 orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id # Burası kalıcı olmuyor buna bak.
#                 df.at[index,'OrcidID_Exact'] = orcid_id
#                 df.at[index,'Match_Type'] = 'Exact'
#                 df.at[index,'NumOfResults'] = 1
#                 exact_count += 1
#                 # print(f"Exact ORCID iD: {orcid_id}")
#             elif 1 < results["num-found"] <= 5:
#                 # Get the first ORCID iD holder from the search results
#                 # orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id
#                 orcid_ids = []
#                 for i in range(results["num-found"]):
#                     orcid_id = results["result"][i]["orcid-identifier"]["path"]
#                     orcid_ids.append(orcid_id)
                    
                    
#                 df.at[index,'OrcidID_Fuzzy'] = orcid_ids
#                 df.at[index,'Match_Type'] = 'Fuzzy'
#                 df.at[index,'NumOfResults'] = results["num-found"]
#                 fuzzy_count += 1
#             elif 5 < results["num-found"] :
#                 # Get the first ORCID iD holder from the search results
#                 # orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id
#                 df.at[index,'OrcidID_Fuzzy'] = np.nan
#                 df.at[index,'Match_Type'] = 'Too_Fuzzy'
#                 df.at[index,'NumOfResults'] = results["num-found"]
#                 too_fuzzy_count += 1
#             else:
#                 no_name_count += 1
#         else:
#             else_count += 1
#             print('Error Message !')
            
            
#     elif type(row['basic_authorized_official_first_name0']) != float :
#         given_name = row['basic_authorized_official_first_name0']
#         family_name = row['basic_authorized_official_last_name0']
#         middle_name = row['basic_authorized_official_middle_name0']
        
        
#         params = {
#             "q": f"family-name:{family_name} AND given-names:{given_name} other-names:{middle_name}" ,
#         }
        
#         # params = {
#         #     # 'q' : A full text search query that searches across multiple fields in the ORCID record, including the researcher's name, title, abstract, and keywords.
#         #     "q": f"given-and-family-names:exact(\"{full_name}\") affiliation-org-name:{affiliation} email:{email} credit-name:{middle_name} other-names:{middle_name}" ,
#         # }

#         response = requests.get(api_endpoint, headers=headers, params=params)
#         results = response.json()
        
#         # Check the status code of the response
#         if response.status_code == 200:
#             # If the request is successful, the search results will be in the response
#             if results["num-found"] == 1:
#                 # Get the only ORCID iD holder from the search results
#                 orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id # Burası kalıcı olmuyor buna bak.
#                 df.at[index,'OrcidID'] = orcid_id
#                 df.at[index,'Match_Type'] = 'Exact'
#                 df.at[index,'NumOfResults'] = 1
#                 exact_count += 1
#                 # print(f"Exact ORCID iD: {orcid_id}")
#             elif 1 < results["num-found"] <= 5:
#                 # Get the first ORCID iD holder from the search results
#                 orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id
#                 df.at[index,'OrcidID'] = orcid_id
#                 df.at[index,'Match_Type'] = 'Fuzzy'
#                 df.at[index,'NumOfResults'] = results["num-found"]
#                 fuzzy_count += 1
#             elif 5 < results["num-found"] :
#                 # Get the first ORCID iD holder from the search results
#                 orcid_id = results["result"][0]["orcid-identifier"]["path"]
#                 # row['OrcidID'] = orcid_id
#                 df.at[index,'OrcidID'] = orcid_id
#                 df.at[index,'Match_Type'] = 'Too_Fuzzy'
#                 df.at[index,'NumOfResults'] = results["num-found"]
#                 too_fuzzy_count += 1
#             else:
#                 no_name_count += 1
#         else:
#             else_count += 1
#             print('Error Message !')
#     else : 
#         no_name_count += 1

In [ ]:
df.head() # OrcidID_Exact ve OrcidID_Fuzzy şeklinde 2 column oluştur.

In [ ]:
total = exact_count + fuzzy_count + too_fuzzy_count

In [ ]:
exact_rate = exact_count / total
fuzzy_rate = fuzzy_count / total
too_fuzzy_rate = too_fuzzy_count / total
no_name_rate = no_name_count / df.shape[0]

In [ ]:
print(f"""
      No Name Count : {no_name_count}
      No Name Rate : {no_name_rate}
      Orcid Name Rate {1-no_name_rate}
      
      --------------------
      Total Count : {total}
      Exact Count : {exact_count} Exact Rate : {exact_rate}
      Fuzzy Count : {fuzzy_count} Fuzzy Rate : {fuzzy_rate} 
      Too Fuzzy Count : {too_fuzzy_count} Fuzzy Rate : {too_fuzzy_rate} 
      """)

In [ ]:
df.to_excel('Fuzzy-names-sample10000.xlsx')

In [ ]:
# Done